As the given dataset in zip file we will unzip the dataset , here i have used zipfile module 


In [ ]:
import zipfile 



i have used numpy,tensorflow,tqdm libraries for my project 

In [27]:
pip install "numpy<2"

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
import os

from tqdm import tqdm

In [3]:

import os
print(os.getcwd())



C:\Users\adity\soulitpageproject


In [5]:
import os

print(os.path.exists("C:/Users/adity/soulitpageproject/recognition"))  # Check folder
print(os.path.exists("C:/Users/adity/soulitpageproject/detection"))  # Check images



True
True


##  Image Preprocessing with TensorFlow

This notebook section loads and preprocesses all images from a folder using TensorFlow.  
It performs:
- Resizing
- Normalization
- Grayscale (for recognition)
- RGB (for detection)


In [8]:
import tensorflow as tf
import os
import numpy as np
from tqdm import tqdm

def preprocess_detection_image(img_path, target_size=(224, 224)):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0
    return img
def preprocess_recognition_image(img_path, target_size=(128, 32)):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size, color_mode='grayscale')
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0
    return img

def preprocess_folder(folder_path, preprocess_fn):
    images = []
    for filename in tqdm(sorted(os.listdir(folder_path))):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, filename)
            img = preprocess_fn(img_path)
            images.append(img)
    return np.array(images)

detection_path = "C:/Users/adity/soulitpageproject/detection/images"
recognition_path ="C:/Users/adity/soulitpageproject/recognition/images"


detection_images = preprocess_folder(detection_path, preprocess_detection_image)
print("✅ Detection Images Shape:", detection_images.shape)

recognition_images = preprocess_folder(recognition_path, preprocess_recognition_image)
print("✅ Recognition Images Shape:", recognition_images.shape)


100%|███████████████████████████████████████████████████████████████████████████████| 901/901 [00:06<00:00, 139.69it/s]


✅ Detection Images Shape: (900, 224, 224, 3)


100%|██████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 1351.27it/s]

✅ Recognition Images Shape: (900, 128, 32, 1)


In [9]:
!pip install ultralytics --upgrade


  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl (12.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.


In [10]:
from ultralytics import YOLO



In [11]:
import pandas as pd

df = pd.read_csv('detection/labels.csv')
print(df.columns.tolist())
print(df.head())



['img_id', 'ymin', 'xmin', 'ymax', 'xmax']
    img_id  ymin  xmin  ymax  xmax
0    1.jpg   276    94   326   169
1   10.jpg   311   395   344   444
2  100.jpg   406   263   450   434
3  101.jpg   283   363   315   494
4  102.jpg   139    42   280   222


In [12]:
import os
import cv2
import pandas as pd

# Load CSV
df = pd.read_csv('detection/labels.csv')
image_dir = 'detection/images'
label_dir = 'detection/labels'

os.makedirs(label_dir, exist_ok=True)

for img_file in df['img_id'].unique():
    img_path = os.path.join(image_dir, img_file)
    
    if not os.path.exists(img_path):
        print(f"❌ Image not found: {img_path}")
        continue

    # Load image to get size
    img = cv2.imread(img_path)
    h, w, _ = img.shape

    # Filter rows for this image
    boxes = df[df['img_id'] == img_file]

    yolo_lines = []
    for _, row in boxes.iterrows():
        # Bounding box
        x_min = row['xmin']
        y_min = row['ymin']
        x_max = row['xmax']
        y_max = row['ymax']

        # Convert to YOLO format
        x_center = (x_min + x_max) / 2.0 / w
        y_center = (y_min + y_max) / 2.0 / h
        box_width = (x_max - x_min) / w
        box_height = (y_max - y_min) / h

        # Only one class — license plate → class id = 0
        yolo_lines.append(f"0 {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")

    # Write label file
    label_path = os.path.join(label_dir, img_file.replace('.jpg', '.txt'))
    with open(label_path, 'w') as f:
        f.write('\n'.join(yolo_lines))


In [13]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  
model.train(data="C:/Users/adity/soulitpageproject/detection/data.yaml", epochs=2, imgsz=640)


Ultralytics 8.3.168  Python-3.12.7 torch-2.7.1+cpu CPU (Intel Core(TM) i5-8250U 1.60GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/adity/soulitpageproject/detection/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train9, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plot

train: Scanning C:\Users\adity\soulitpageproject\detection\labels.cache... 900 images, 0 backgrounds, 0 corrupt: 100%|█

val: Fast image access  (ping: 0.10.0 ms, read: 229.0123.5 MB/s, size: 117.2 KB)



C:\Users\adity\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\adity\soulitpageproject\detection\labels.cache... 900 images, 0 backgrounds, 0 corrupt: 100%|███
C:\Users\adity\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to runs\detect\train9\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train9
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        2/2         0G      1.061        1.3     0.9829          9        640: 100%|██████████| 57/57 [07:36<00:00,  8.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [01:54


                   all        900        900      0.975      0.981      0.994      0.719

2 epochs completed in 0.321 hours.
Optimizer stripped from runs\detect\train9\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train9\weights\best.pt, 6.2MB

Validating runs\detect\train9\weights\best.pt...
Ultralytics 8.3.168  Python-3.12.7 torch-2.7.1+cpu CPU (Intel Core(TM) i5-8250U 1.60GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [01:44


                   all        900        900      0.975      0.981      0.994       0.72
Speed: 2.2ms preprocess, 91.8ms inference, 0.0ms loss, 10.6ms postprocess per image
Results saved to runs\detect\train9


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000025ED9647740>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [14]:
metrics = model.val()  
print(metrics)  #  mAP, precision, recall


Ultralytics 8.3.168  Python-3.12.7 torch-2.7.1+cpu CPU (Intel Core(TM) i5-8250U 1.60GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 241.251.9 MB/s, size: 95.0 KB)


val: Scanning C:\Users\adity\soulitpageproject\detection\labels.cache... 900 images, 0 backgrounds, 0 corrupt: 100%|███
C:\Users\adity\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 57/57 [01:28


                   all        900        900      0.975      0.983      0.994      0.719
Speed: 1.3ms preprocess, 75.6ms inference, 0.0ms loss, 9.4ms postprocess per image
Results saved to runs\detect\train92
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000025EE749DB20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,   

In [15]:
from ultralytics import YOLO
import cv2
import os


model = YOLO("soulitpageproject/yolov8n.pt")  

test_folder = r"C:\Users\adity\soulitpageproject\test-Copy1"
output_folder = "cropped_plates"
os.makedirs(output_folder, exist_ok=True)


supported_ext = ('.jpg', '.jpeg', '.png')


for filename in os.listdir(test_folder):
    if filename.lower().endswith(supported_ext):
        image_path = os.path.join(test_folder, filename)
        img = cv2.imread(image_path)

        
        results = model.predict(image_path)

        for r in results:
            boxes = r.boxes.xyxy.cpu().numpy()
            for i, (x1, y1, x2, y2) in enumerate(boxes):
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                crop = img[y1:y2, x1:x2]
                
                
                crop_filename = f"{os.path.splitext(filename)[0]}_plate_{i}.jpg"
                cv2.imwrite(os.path.join(output_folder, crop_filename), crop)




image 1/1 C:\Users\adity\soulitpageproject\test-Copy1\1000.jpg: 448x640 1 person, 1 bicycle, 1 car, 126.8ms
Speed: 3.8ms preprocess, 126.8ms inference, 1.8ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 C:\Users\adity\soulitpageproject\test-Copy1\1001.jpg: 640x384 1 car, 124.4ms
Speed: 3.0ms preprocess, 124.4ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 C:\Users\adity\soulitpageproject\test-Copy1\1002.jpg: 640x480 1 car, 1 sheep, 143.0ms
Speed: 4.6ms preprocess, 143.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Users\adity\soulitpageproject\test-Copy1\1003.jpg: 384x640 1 car, 144.4ms
Speed: 3.9ms preprocess, 144.4ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\adity\soulitpageproject\test-Copy1\1004.jpg: 640x384 2 cars, 121.3ms
Speed: 4.8ms preprocess, 121.3ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 C:\Users\adity\soulitpagepro

In [16]:
!pip install easyocr


In [20]:
from ultralytics import YOLO
import cv2
import os

# Load the trained model
model = YOLO("soulitpageproject/yolov8n.pt")  # update as needed

# Create a folder to save cropped plates
os.makedirs("cropped_plates", exist_ok=True)

# Folder of test images
test_folder = "C:/Users/adity/soulitpageproject/test-Copy1"

# Loop over all images in the folder
for filename in os.listdir(test_folder):
    if filename.lower().endswith((".jpg", ".png", ".jpeg")):
        image_path = os.path.join(test_folder, filename)
        img = cv2.imread(image_path)
        results = model.predict(image_path)

        for r in results:
            boxes = r.boxes.xyxy.cpu().numpy()
            for i, box in enumerate(boxes):
                x1, y1, x2, y2 = map(int, box[:4])  # Fix here
                crop = img[y1:y2, x1:x2]
                save_path = f"cropped_plates/{filename[:-4]}_plate_{i}.jpg"
                cv2.imwrite(save_path, crop)




image 1/1 C:\Users\adity\soulitpageproject\test-Copy1\1000.jpg: 448x640 1 person, 1 bicycle, 1 car, 93.4ms
Speed: 4.3ms preprocess, 93.4ms inference, 1.6ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 C:\Users\adity\soulitpageproject\test-Copy1\1001.jpg: 640x384 1 car, 83.1ms
Speed: 3.2ms preprocess, 83.1ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 C:\Users\adity\soulitpageproject\test-Copy1\1002.jpg: 640x480 1 car, 1 sheep, 98.6ms
Speed: 4.2ms preprocess, 98.6ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Users\adity\soulitpageproject\test-Copy1\1003.jpg: 384x640 1 car, 90.0ms
Speed: 3.3ms preprocess, 90.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\adity\soulitpageproject\test-Copy1\1004.jpg: 640x384 2 cars, 92.7ms
Speed: 3.0ms preprocess, 92.7ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 C:\Users\adity\soulitpageproject\test-

In [22]:
import easyocr
import os
import cv2


reader = easyocr.Reader(['en'])
cropped_folder = "cropped_plates"


for filename in os.listdir(cropped_folder):
    if filename.lower().endswith((".jpg", ".png", ".jpeg")):
        image_path = os.path.join(cropped_folder, filename)
        image = cv2.imread(image_path)

        
        result = reader.readtext(image)

        print(f"\n📄 OCR results for {filename}:")
        for bbox, text, conf in result:
            print(f"Detected Text: {text} | Confidence: {conf:.2f}")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



📄 OCR results for 1000_plate_0.jpg:


C:\Users\adity\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



📄 OCR results for 1000_plate_1.jpg:
Detected Text: DOLO | Confidence: 0.58
Detected Text: EL] | Confidence: 0.29
Detected Text: Lns | Confidence: 0.01
Detected Text: [94 | Confidence: 0.96
Detected Text: Un; | Confidence: 0.36
Detected Text: 573] | Confidence: 0.68
Detected Text: wwWPOlo Glassic | Confidence: 0.11
Detected Text: OOm | Confidence: 0.07

📄 OCR results for 1000_plate_2.jpg:

📄 OCR results for 1001_plate_0.jpg:
Detected Text: 1733 59116 | Confidence: 0.28

📄 OCR results for 1002_plate_0.jpg:
Detected Text: 1524 | Confidence: 0.83

📄 OCR results for 1002_plate_1.jpg:

📄 OCR results for 1003_plate_0.jpg:
Detected Text: 92  } 3039 | Confidence: 0.34

📄 OCR results for 1004_plate_0.jpg:
Detected Text: 76 | Confidence: 1.00
Detected Text: UjS | Confidence: 0.18
Detected Text: 2714 | Confidence: 0.94

📄 OCR results for 1004_plate_1.jpg:

📄 OCR results for 1005_plate_0.jpg:
Detected Text: N9La ; 558 | Confidence: 0.01

📄 OCR results for 1005_plate_1.jpg:

📄 OCR results for 1005_

In [48]:
import os
import cv2
import easyocr

reader = easyocr.Reader(['en'], gpu=False)

cropped_folder = r"C:\Users\adity\soulitpageproject\cropped_plates"

for img_name in os.listdir(cropped_folder):
    img_path = os.path.join(cropped_folder, img_name)

    img = cv2.imread(img_path)
    if img is None:
        print(f"❌ Skipping unreadable image: {img_path}")
        continue

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = reader.readtext(img_rgb, detail=0)

    detected_text = " ".join(result)
    print(f"{img_name}: {detected_text}")



Using CPU. Note: This module is much faster with a GPU.


1000_plate_0.jpg: 


C:\Users\adity\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


1000_plate_1.jpg: DOLO EL] Clad I[94 On; 573] WwWPolo Glasbic OOm
1000_plate_2.jpg: 
1001_plate_0.jpg: 1730 59116
1002_plate_0.jpg: 1524 35
1002_plate_1.jpg: 
1003_plate_0.jpg: 92  } 3039
1004_plate_0.jpg: 76 2714
1004_plate_1.jpg: 
1005_plate_0.jpg: N94a ; 558
1005_plate_1.jpg: 
1005_plate_2.jpg: 
1005_plate_3.jpg: 
1006_plate_0.jpg: 180 033 228
1006_plate_1.jpg: 
1006_plate_2.jpg: 
1007_plate_0.jpg: [1b 16070
1008_plate_0.jpg: 104 3, 9416
1008_plate_1.jpg: 
1009_plate_0.jpg: 
1009_plate_1.jpg: 
1009_plate_2.jpg: 246451398
1010_plate_0.jpg: 41276314029
1011_plate_0.jpg: 
1012_plate_0.jpg: 49
1013_plate_0.jpg: 10z*59648
1013_plate_1.jpg: 
1014_plate_0.jpg: 49651332]
1014_plate_1.jpg: 
1014_plate_2.jpg: 
1014_plate_3.jpg: 
1015_plate_0.jpg: 95 > 1J0L
1015_plate_1.jpg: 
1016_plate_0.jpg: 1433 , 4790
1016_plate_1.jpg: 
1016_plate_2.jpg: 
1017_plate_0.jpg: VECTRA 20 DrI [1828*361497 OPEL
1018_plate_0.jpg: 88 Ui 6386 _
1018_plate_1.jpg: 
1018_plate_2.jpg: 
1019_plate_0.jpg: 4158 '+;4820
102

In [52]:
import os
import pandas as pd
import cv2
import easyocr

# 1. Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=False)

# 2. Path to cropped images
cropped_folder = r"C:\Users\adity\soulitpageproject\cropped_plates"

# 3. Prepare list to collect results
data = []

# 4. Process each image
for img_name in sorted(os.listdir(cropped_folder)):
    img_path = os.path.join(cropped_folder, img_name)
    
    # Read and preprocess image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Skipping unreadable image: {img_name}")
        continue
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # OCR detection
    result = reader.readtext(img_rgb, detail=0)
    detected_text = ''.join(result)
    
    # Create digit presence vector (for digits 0 to 5 only)
    digit_vector = [0]*6  # Only digits 0-5
    for ch in detected_text:
        if ch.isdigit():
            digit = int(ch)
            if 0 <= digit <= 5:
                digit_vector[digit] = 1
    
    # Add row to data
    row = [os.path.splitext(img_name)[0]] + digit_vector
    data.append(row)

# 5. Create DataFrame with specified columns
columns = ['id'] + [str(i) for i in range(6)]
df = pd.DataFrame(data, columns=columns)

# 6. Save to CSV
df.to_csv("final_submission.csv", index=False)
print("✅ CSV saved as 'final_submission.csv'")



Using CPU. Note: This module is much faster with a GPU.


✅ CSV saved as 'final_submission.csv'
